In [ ]:
from pathlib import Path
import re
from openai import OpenAI
from sqlalchemy import inspect as sqlalchemy_inspect

# -------------------------------
# Setup OpenAI client
# -------------------------------
client = OpenAI(api_key="")


# -------------------------------
# Helper utilities
# -------------------------------
def clean_code(text: str) -> str:
    """Remove markdown-style fences like ```python or ```html."""
    return re.sub(r"```[a-zA-Z]*\n?|```", "", text).strip()

def ask_gpt(prompt: str) -> str:
    """Send a prompt to GPT and return clean code."""
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are an expert FastAPI full-stack developer. Return only raw code, no markdown."},
            {"role": "user", "content": prompt},
        ],
        #temperature=0.4,
    )
    return clean_code(response.choices[0].message.content)

# -------------------------------
# Step 1: Analyze app idea
# -------------------------------
def analyze_prompt(user_prompt: str) -> str:
    print(f"🧠 Analyzing app idea: {user_prompt}")
    analysis_prompt = f"""
    The user wants a database-backed app.
    Describe the entities, fields, and relationships.

    User description:
    {user_prompt}
    """
    analysis = ask_gpt(analysis_prompt)
    print("\n📋 Schema plan:\n", analysis)
    return analysis

# -------------------------------
# Step 2: Generate schema.py
# -------------------------------
def generate_schema(analysis: str):
    print("\n🏗️ Generating SQLAlchemy schema...")
    prompt = f"""
    Write valid Python code defining SQLAlchemy 2.0 ORM models using DeclarativeBase.

    Requirements:
    - Define class Base(DeclarativeBase)
    - Import from sqlalchemy.orm: DeclarativeBase, Mapped, mapped_column, relationship
    - Import from sqlalchemy: Integer, String, Text, DateTime, ForeignKey
    - Include __allow_unmapped__ = True in each class
    - Each model must define a unique __tablename__
    - Use mapped_column instead of Column
    - Include relationships where appropriate
    - When a table has multiple foreign keys to the same parent, specify `foreign_keys` explicitly in relationship()
    - Avoid using reserved names like 'metadata', 'query', or 'registry'
    - Use 'meta_data' instead of 'metadata'
    - No markdown or explanations

    Schema description:
    {analysis}
    """
    code = ask_gpt(prompt)
    Path("generated/schema.py").write_text(code)
    print("✅ Saved generated/schema.py")


# -------------------------------
# Step 3: Extract model names
# -------------------------------
def extract_model_names():
    code = Path("generated/schema.py").read_text()
    return re.findall(r"class\s+(\w+)\s*\(Base\)", code)

# -------------------------------
# Step 4: Get model fields dynamically
# -------------------------------
def get_model_fields(model_name):
    """Return dict of model fields for dynamic Form() generation."""
    from generated import schema
    model = getattr(schema, model_name)
    mapper = sqlalchemy_inspect(model)
    return {col.key: col.type.python_type.__name__ for col in mapper.columns if col.key != "id"}

# -------------------------------
# Step 5: Generate backend API
# -------------------------------

def generate_api():
    """
    Generates a FastAPI backend (generated/api.py) with form-compatible endpoints.
    This ensures HTMX forms (which send Form data) always match backend expectations.
    """
    print("\n⚙️ Generating FastAPI backend...")

    from generated import schema
    import inspect, re
    from pathlib import Path

    # Dynamically extract model classes from schema
    models = [
        name for name, obj in inspect.getmembers(schema)
        if inspect.isclass(obj) and hasattr(obj, "__tablename__")
    ]

    # Build safe import line
    imports = ", ".join(["Base"] + models)

    api_code = f"""from fastapi import FastAPI, HTTPException, Form, Depends
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine

try:
    from .schema import {imports}
except ImportError:
    from .schema import Base

SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={{"check_same_thread": False}})
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

app = FastAPI(title="Generated API", version="1.0")

# CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:8001", "http://localhost:8001", "http://127.0.0.1", "http://localhost"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

Base.metadata.create_all(bind=engine)
"""

    # helper: convert CamelCase → snake_case
    def snake_case(name):
        return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

    # generate CRUD routes
    for model_name in models:
        endpoint = snake_case(model_name)
        api_code += f"""

@app.post("/{endpoint}/")
async def create_{endpoint}(
"""
        # dynamically introspect model columns
        model_cls = getattr(schema, model_name)
        params = []
        assignments = []
        for col_name, col_type in model_cls.__annotations__.items():
            if col_name == "id":
                continue
            # default all to Form(...) for HTMX compatibility
            params.append(f"    {col_name}: str = Form(...)")
            assignments.append(f"{col_name}={col_name}")

        api_code += ",\n".join(params) + ",\n    db: Session = Depends(get_db)\n):\n"
        api_code += f"    new_item = {model_name}({', '.join(assignments)})\n"
        api_code += "    db.add(new_item)\n    db.commit()\n    db.refresh(new_item)\n    return new_item\n"

        # READ
        api_code += f"""
@app.get("/{endpoint}/")
async def read_{endpoint}s(db: Session = Depends(get_db)):
    return db.query({model_name}).all()
"""

        # DELETE
        api_code += f"""
@app.delete("/{endpoint}/{{item_id}}")
async def delete_{endpoint}(item_id: int, db: Session = Depends(get_db)):
    item = db.query({model_name}).filter({model_name}.id == item_id).first()
    if not item:
        raise HTTPException(status_code=404, detail="{model_name} not found")
    db.delete(item)
    db.commit()
    return {{"message": "{model_name} deleted"}}
"""

    Path("generated/api.py").write_text(api_code)
    print("✅ API generated successfully with HTMX-compatible Form endpoints!")



# -------------------------------
# Step 6: Generate frontend.py (fixed)
# -------------------------------
def generate_frontend(models: list):
    print("\n🎨 Generating frontend app...")
    model_imports = ", ".join(models)
    prompt = f"""
Write valid Python code for a FastAPI app named 'app' that:
- Imports Base, {model_imports} from .schema
- Imports get_db from .api
- Uses Jinja2Templates("./generated/templates")
- Renders index.html on '/'
- Includes CRUD endpoints using Form() params and RedirectResponse
- Does NOT use response_model or ORM types in function annotations
- Returns RedirectResponse after POST and {{}} dicts for delete
- All routes must include response_model=None in their decorators
- No markdown fences
"""
    code = ask_gpt(prompt)
    Path("generated/frontend.py").write_text(code)
    print("✅ Saved generated/frontend.py")

# -------------------------------
# Step 7: Generate index.html
# ------------------------------
def generate_index_html(models: list):
    print("\n🧱 Generating index.html...")
    Path("generated/templates").mkdir(parents=True, exist_ok=True)
    prompt = f"""
Write a simple responsive Jinja2 HTML page using TailwindCSS and HTMX
for models {models}. It should:
- Display data tables and forms for each model
- Use hx-get, hx-post, and hx-delete for CRUD operations
- Target endpoints on http://127.0.0.1:8000
- Include a navbar and reload sections dynamically
- No markdown or code fences
"""
    html = ask_gpt(prompt)
    Path("generated/templates/index.html").write_text(html)
    print("✅ Saved generated/templates/index.html")

# -------------------------------
# Step 8: Main orchestration
# -------------------------------
def main():
    user_prompt = input("🗣️ Describe your app: ")
    Path("generated").mkdir(exist_ok=True)
    Path("generated/__init__.py").touch(exist_ok=True)

    analysis = analyze_prompt(user_prompt)
    generate_schema(analysis)
    models = extract_model_names()
    generate_api()
    generate_frontend(models)
    generate_index_html(models)

    print("\n🎉 All components generated successfully!")
    print("➡️ Run backend:  uvicorn generated.api:app --reload --port 8000")
    print("➡️ Run frontend: uvicorn generated.frontend:app --reload --port 8001")
    print("Then open http://127.0.0.1:8001 🚀")

if __name__ == "__main__":
    main()


🗣️ Describe your app:  an application to manage users feedback


🧠 Analyzing app idea: an application to manage users feedback
